In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import keras_tuner as kt

from numpy import array
from numpy import hstack
from numpy import empty
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from os import chdir
from os import getcwd
from os import listdir
import math

csv_file_path = 'D:/Arbeit PhD/Fachlich/z_Sonstiges/Groundwater challenge/data/Sweden_1'
chdir(csv_file_path)

Y_all = pd.read_csv('heads.csv',decimal='.',index_col=0, delimiter=',', header=0,parse_dates=True)
X_all = pd.read_csv('input_data.csv',decimal='.',index_col='time', delimiter=',', header=0,parse_dates=True)

Y = Y_all['2001-02-06':' 2015-12-29']
X = X_all['2001-01-07':' 2015-12-29']
# for sweden weekly forecasts are asked
print(X.index, Y.index)

DatetimeIndex(['2001-01-07', '2001-01-08', '2001-01-09', '2001-01-10',
               '2001-01-11', '2001-01-12', '2001-01-13', '2001-01-14',
               '2001-01-15', '2001-01-16',
               ...
               '2015-12-20', '2015-12-21', '2015-12-22', '2015-12-23',
               '2015-12-24', '2015-12-25', '2015-12-26', '2015-12-27',
               '2015-12-28', '2015-12-29'],
              dtype='datetime64[ns]', name='time', length=5470, freq=None) DatetimeIndex(['2001-02-06', '2001-02-13', '2001-02-20', '2001-02-27',
               '2001-03-06', '2001-03-13', '2001-03-20', '2001-03-27',
               '2001-04-03', '2001-04-10',
               ...
               '2015-10-27', '2015-11-03', '2015-11-10', '2015-11-17',
               '2015-11-24', '2015-12-01', '2015-12-08', '2015-12-15',
               '2015-12-22', '2015-12-29'],
              dtype='datetime64[ns]', name='Date', length=778, freq=None)


In [2]:
X_train = X[:'2011-12-31']
X_test = X['2014-01-01':]
X_valid = X['2012-01-01':'2013-12-31']

Y_train = Y[:'2011-12-31']
Y_test = Y['2014-01-01':]
Y_valid = Y['2012-01-01':'2013-12-31']


csv_file_path = 'D:\Arbeit PhD\Fachlich\z_Sonstiges\Groundwater challenge\data'
chdir(csv_file_path)
from helper import *

(x_tr,y, x_te, y_test, x_va, y_valid, scaler_X,scaler_Y)=prepare_data_weekly(X_train,Y_train,X_valid,Y_valid,X_test,Y_test)
print(x_te.shape)
print(x_tr.shape)
print(x_va.shape)

iters=3
n_steps_in=30
n_steps_out=1

x_final_unsplit = scaler_X.transform(X)
y_final = scaler_Y.transform(Y)

x_final = split_sequences_x(x_final_unsplit, n_steps_in, n_steps_out)
print(x_final.shape, y_final.shape) # 5282
5440/7

(569, 1) y_train.shape transformed
(728, 9)
(4011, 9)
(731, 9)
(778, 30, 9) (778, 1)


777.1428571428571

In [3]:

import keras_tuner as kt
from tensorflow.keras.layers import Dropout
from tensorflow.keras import backend as K
class Dropout(keras.layers.Dropout):
    def __init__(self, rate, training=None, noise_shape=None, seed=None, **kwargs):
        super(Dropout, self).__init__(rate, noise_shape=None, seed=None,**kwargs)
        self.training = training

        
    def call(self, inputs, training=None):
        if 0. < self.rate < 1.:
            noise_shape = self._get_noise_shape(inputs)

            def dropped_inputs():
                return K.dropout(inputs, self.rate, noise_shape,
                                 seed=self.seed)
            if not training: 
                return K.in_train_phase(dropped_inputs, inputs, training=self.training)
            return K.in_train_phase(dropped_inputs, inputs, training=training)
        return inputs

In [4]:
import keras_tuner as kt
from tensorflow.keras.layers import Dropout
from tensorflow.keras import backend as K

class Dropout(keras.layers.Dropout):
    def __init__(self, rate, training=None, noise_shape=None, seed=None, **kwargs):
        super(Dropout, self).__init__(rate, noise_shape=None, seed=None,**kwargs)
        self.training = training

        
    def call(self, inputs, training=None):
        if 0. < self.rate < 1.:
            noise_shape = self._get_noise_shape(inputs)

            def dropped_inputs():
                return K.dropout(inputs, self.rate, noise_shape,
                                 seed=self.seed)
            if not training: 
                return K.in_train_phase(dropped_inputs, inputs, training=self.training)
            return K.in_train_phase(dropped_inputs, inputs, training=training)
        return inputs

In [5]:
test_runs = 100
    
#load models 
csv_file_path = 'D:\Arbeit PhD\Fachlich\z_Sonstiges\Groundwater challenge'
chdir(csv_file_path)

transformer_model = keras.models.load_model('TF_Sweden1_tune.h5')
transformer_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 30, 9)]      0                                            
__________________________________________________________________________________________________
layer_normalization (LayerNorma (None, 30, 9)        18          input_1[0][0]                    
__________________________________________________________________________________________________
multi_head_attention (MultiHead (None, 30, 9)        1413        layer_normalization[0][0]        
                                                                 layer_normalization[0][0]        
__________________________________________________________________________________________________
dropout (Dropout)               (None, 30, 9)        0           multi_head_attention[0][0]   

## Prediction Transformer

In [6]:

print(x_final.shape)
Y_pred_test_tr = np.empty((x_final.shape[0],n_steps_out,test_runs))
y_correct_utf = scaler_Y.inverse_transform(y_final)
for j in range(test_runs): 
    Y_pred_test_tr[:,:,j]=transformer_model.predict(x_final)
    yhat_utf = scaler_Y.inverse_transform(Y_pred_test_tr[:,:,j])
    

(778, 30, 9)


## POC

In [7]:
print(Y_pred_test_tr.shape)
Y_pred = np.mean(Y_pred_test_tr, axis=2)
Y_std = np.std(Y_pred_test_tr, axis=2)
zT = 6.4
T_ival = [Y_pred - zT*Y_std,Y_pred + zT*Y_std]
print(T_ival[0].shape, y_final.shape)
#print(range(len(Y_test_day1)))
def calc_POC(list_of_bounds, measured):
    temp = 0 
    for i in range(len(measured)):
        if measured[i]< list_of_bounds[1][i] and measured[i]> list_of_bounds[0][i]:
            temp = temp + 1
    POC = temp/len(measured)
    return POC

print(T_ival[1].shape)
POC_tr = calc_POC(T_ival,y_final)
print(POC_tr)

(778, 1, 100)
(778, 1) (778, 1)
(778, 1)
0.974293059125964


In [8]:
print(Y_pred_test_tr.shape)
Y_pred_test_tr_flat = Y_pred_test_tr.reshape((test_runs*Y_pred_test_tr.shape[0], 1))
Y_pred_test_utf_flat = scaler_Y.inverse_transform(Y_pred_test_tr_flat)
Y_pred_test_utf = Y_pred_test_utf_flat.reshape((Y_pred_test_tr.shape[0], 1,test_runs))
Y_pred_calib = np.mean(Y_pred_test_utf, axis=2)
Y_std_calib = np.std(Y_pred_test_utf, axis=2)
T_ival_calib = [Y_pred_calib - zT*Y_std_calib,Y_pred_calib + zT*Y_std_calib]
#T_ival_calib[1]

(778, 1, 100)


In [9]:
csv_file_path = 'D:/Arbeit PhD/Fachlich/z_Sonstiges/Groundwater challenge/data/Sweden_1'
chdir(csv_file_path)

X_all = pd.read_csv('input_data.csv',decimal='.',index_col='time', delimiter=',', header=0,parse_dates=True)

X_finaltest = X_all['2016-01-01':]
X_finaltest

,rr,tg,tn,tx,pp,hu,fg,qq,et
time,,,,,,,,,
2016-01-01,0.0,-2.170000,-3.110000,-0.45,1032.60000,85.625000,3.35,1.0,0.008064
2016-01-02,0.0,-4.370000,-5.270000,-2.94,1041.10000,86.725000,1.80,1.0,0.007273
2016-01-03,1.5,-9.290000,-11.870000,-4.79,1030.00000,88.626320,1.06,1.0,0.005744
2016-01-04,0.0,-20.510000,-24.430000,-8.88,1025.60000,84.445000,1.37,1.0,0.002913
2016-01-05,0.0,-20.820000,-24.029999,-15.88,1018.60004,80.870000,0.88,1.0,0.002869
...,...,...,...,...,...,...,...,...,...
2021-12-27,0.0,-16.869999,-20.789999,-12.86,1006.90000,72.770004,0.27,1.0,0.003763
2021-12-28,3.6,-18.359999,-23.800000,-14.53,1007.90000,79.075005,1.02,1.0,0.003416
2021-12-29,2.1,-10.690000,-13.830000,-9.55,1001.30000,80.955000,0.73,1.0,0.005442


In [10]:
X_finaltest_trf = scaler_X.transform(X_finaltest)
X_ftest = split_sequences_x(X_finaltest_trf,n_steps_in, n_steps_out)
X_ftest.shape

(309, 30, 9)

In [11]:
print(x_final.shape)
Y_pred_ftest_tr = np.empty((X_ftest.shape[0],n_steps_out,test_runs))
for j in range(test_runs): 
    Y_pred_ftest_tr[:,:,j]=transformer_model.predict(X_ftest)

(778, 30, 9)


In [12]:
Y_pred_ftest_tr_flat = Y_pred_ftest_tr.reshape((100*Y_pred_ftest_tr.shape[0], 1))
Y_pred_ftest_utf_flat = scaler_Y.inverse_transform(Y_pred_ftest_tr_flat)
Y_pred_ftest_utf = Y_pred_ftest_utf_flat.reshape((Y_pred_ftest_tr.shape[0], 1,100))

yhat_mean = np.mean(Y_pred_ftest_utf, axis=2)
yhat_std = np.std(Y_pred_ftest_utf, axis=2)
T_ival_utf = [yhat_mean - zT*yhat_std,yhat_mean + zT*yhat_std]
yhat_mean.shape

(309, 1)

In [13]:
final_idx_calib = pd.date_range(start='2001-02-06', end='2015-12-29', freq='7d')
print(final_idx_calib)
final_idx_test = pd.date_range(start='2016-01-31', end='2021-12-31', freq='7d')
final_df_calib = pd.DataFrame(index=final_idx_calib)
final_df_test = pd.DataFrame(index=final_idx_test)
final_df_test.shape

DatetimeIndex(['2001-02-06', '2001-02-13', '2001-02-20', '2001-02-27',
               '2001-03-06', '2001-03-13', '2001-03-20', '2001-03-27',
               '2001-04-03', '2001-04-10',
               ...
               '2015-10-27', '2015-11-03', '2015-11-10', '2015-11-17',
               '2015-11-24', '2015-12-01', '2015-12-08', '2015-12-15',
               '2015-12-22', '2015-12-29'],
              dtype='datetime64[ns]', length=778, freq='7D')


(309, 0)

In [14]:
# Y_pred und T_ival können schon eingelesen werden in den dataframe von begin
print(Y_pred_calib.shape, final_df_calib.shape)
final_df_calib['head forecast'] = Y_pred_calib
final_df_calib['95% lower'] = T_ival_calib[0]
final_df_calib['95% upper'] = T_ival_calib[1]
final_df_calib

(778, 1) (778, 0)


,head forecast,95% lower,95% upper
2001-02-06,240.816489,240.308909,241.324068
2001-02-13,240.776042,240.208636,241.343449
2001-02-20,240.697887,240.093512,241.302261
2001-02-27,240.708704,240.163831,241.253577
2001-03-06,240.744647,240.170220,241.319073
...,...,...,...
2015-12-01,241.145940,240.285989,242.005892
2015-12-08,241.159236,240.444503,241.873968
2015-12-15,241.026821,240.372690,241.680952
2015-12-22,241.002247,240.292235,241.712259


In [15]:
final_df_test['head forecast'] = yhat_mean
final_df_test['95% lower'] = T_ival_utf[0]
final_df_test['95% upper'] = T_ival_utf[1]
final_df_test

,head forecast,95% lower,95% upper
2016-01-31,240.691159,240.115693,241.266624
2016-02-07,240.753920,240.157262,241.350578
2016-02-14,240.857161,240.221762,241.492560
2016-02-21,240.922327,240.240453,241.604201
2016-02-28,240.943532,240.316628,241.570435
...,...,...,...
2021-11-28,240.919249,240.240319,241.598179
2021-12-05,240.799912,240.205384,241.394439
2021-12-12,240.679493,240.109726,241.249260
2021-12-19,240.647784,240.167607,241.127960


In [16]:
csv_file_path = 'D:/Arbeit PhD/Fachlich/z_Sonstiges/Groundwater challenge/submissions'
chdir(csv_file_path)
final_df_calib.to_csv('Sweden1_calib.csv', float_format='%.2f')
final_df_test.to_csv('Sweden1_test.csv', float_format='%.2f')